In [1]:
using JuMP, Gurobi

function solveMaze(numWayPoints, numHoverCrafts, numObstacles, A, bo, inputWayPoints, avoidObstacles
    , avoidHoverCraftsCollision, lambda)
    
    m = Model(solver = GurobiSolver(OutputFlag=0))

    # Variable for resulting position for each hovercraft
    @variable(m, x[1:2,1:T[numWayPoints],1:numHoverCrafts])

    # Variable for resulting velocity for each hovercraft
    @variable(m, v[1:2,1:T[numWayPoints],1:numHoverCrafts])

    # Variable for thruster input for each hovercraft
    @variable(m, u[1:2,1:T[numWayPoints],1:numHoverCrafts])

    # Binary variables for logical constraints to be imposed to implement
    # static obstacle avoidance. This is a four dimensional variable where
    # first dimension: number of obstacles
    # second dimension: 1 each for x_min, x_max, y_min and y_max for an obstacle
    # third dimension: Number of time steps.
    # fourth dimension: Number of hovercrafts
    @variable(m, obstacle[1:numObstacles,1:4,1:T[numWayPoints],1:numHoverCrafts], Bin)

    # Binary variables for logical constraints to be imposed to implement
    # collision avoidance between hovercrafts. This is a four dimensional variable where
    # first dimension: 1 each for x_min, x_max, y_min and y_max for an obstacle
    # second dimension: Number of time steps.
    # third dimension: Number of hovercrafts.
    # fourth dimension: Number of hovercrafts.
    @variable(m, obstacleS[1:4,1:T[numWayPoints],1:numHoverCrafts,1:numHoverCrafts], Bin)
    
    vMax = 1
    
    # satisfy the dynamics (with zero initial velocity)
    @constraint(m, v[:,1,:] .== [0;0])

    # Simple linear model of hovercraft dynamics imposed on each timestep.
    # Change in resulting position should be equal to the velocity at that instant.
    # Change in resulting velocity should be equal to the thruster input at that instant.
    for t in 1:T[numWayPoints]-1
        @constraint(m, x[:,t+1,:] .== x[:,t,:] + v[:,t,:])
        @constraint(m, v[:,t+1,:] .== v[:,t,:] + u[:,t,:])
    end

    # Constraint on each hovercraft for each time step to be in the maximum velocity
    # limit.
    for n in 1:numHoverCrafts
        for t in 1:T[numWayPoints]
            @constraint(m, sum(v[:,t,n].^2) <= vMax)
        end
    end
    
    # hit all the waypoints
    for n in 1:numHoverCrafts
        for t in [1 numWayPoints]
            @constraint(m, x[:,T[t],n] .== inputWayPoints[:,t,n])
        end
    end

    # Avoid all obstacles
    if avoidObstacles
        for n in 1:numHoverCrafts
            for o in 1:numObstacles
                for t in 1:T[numWayPoints]-1
                    @constraint(m, A*x[:,t,n] .<= bo[:,o] .+ 1000*obstacle[o,:,t,n])
                    # Constraint to relax no more than three constraints on the resulting position
                    # of each hovercraft. This ensures that all hovercrafts stay out of the bounding
                    # box of each obstacle.
                    @constraint(m, sum(obstacle[o,i,t,n] for i in 1:4) <= 3)
                end
            end
        end
    end
                
    if avoidHoverCraftsCollision
        for n1 in 1:(numHoverCrafts-1)
            for n2 in (n1+1):numHoverCrafts
                for t in 3:T[numWayPoints]-3
                    @constraint(m, x[1,t,n1] <= x[1,t,n2] - .5 + 1000*obstacleS[1,t,n1,n2])
                    @constraint(m, -x[1,t,n1] <= -(x[1,t,n2] + .5) + 1000*obstacleS[2,t,n1,n2])
                    @constraint(m, x[2,t,n1] <= x[2,t,n2] - .5 + 1000*obstacleS[3,t,n1,n2])
                    @constraint(m, -x[2,t,n1] <= -(x[2,t,n2] + .5) + 1000*obstacleS[4,t,n1,n2])
                    @constraint(m, sum(obstacleS[i,t,n1,n2] for i in 1:4) <= 3)
                end
            end
        end
    end

    # minimize 2-norm (THIS IS LEAST-SQUARES)
    @objective(m, Min, sum(u.^2) + lambda*sum(sum((x[:,T[2:numWayPoints],n]-inputWayPoints[:,2:numWayPoints,n]).^2 for n in 1:numHoverCrafts)))
    solve(m)
    xopt=getvalue(x)
    return xopt, getvalue(u), getobjectivevalue(m), getvalue(sum(u.^2)), getvalue(sum(sum((x[:,T[2:numWayPoints],n]-inputWayPoints[:,2:numWayPoints,n]).^2 for n in 1:numHoverCrafts)))
end

solveMaze (generic function with 1 method)

In [2]:
using PyPlot

function plotMazeSolution(xopt, numWayPoints, numHoverCrafts, numObstacles, A, bo, inputWayPoints, xo)
    figure(figsize=(12,4))
    for n in 1:numHoverCrafts
        plot( xopt[1,:,n][:], xopt[2,:,n][:],".-", markersize=4 )
        plot( xopt[1,1:5:T[numWayPoints],n][:], xopt[2,1:5:T[numWayPoints],n][:],"k.", markersize=4 )
    end

    for n in 1:numHoverCrafts
        plot(inputWayPoints[1,:,n], inputWayPoints[2,:,n], "r.", markersize=12 )
    end

    for o in 1:numObstacles
        # lower and upper lines
        plot(linspace(xo[1,o] - bbo[1,o],xo[1,o] + bbo[1,o],10),
            0.*linspace(xo[1,o] - bbo[1,o],xo[1,o] + bbo[1,o],10) .+ (xo[2,o] - bbo[2,o]))
        plot(linspace(xo[1,o] - bbo[1,o],xo[1,o] + bbo[1,o],10),
            0.*linspace(xo[1,o] - bbo[1,o],xo[1,o] + bbo[1,o],10) .+ (xo[2,o] + bbo[2,o]))

        # left and right lines
        plot(0.*linspace(xo[2,o] - bbo[2,o],xo[2,o] + bbo[2,o],10) .+ (xo[1,o] - bbo[1,o]),
            linspace(xo[2,o] - bbo[2,o],xo[2,o] + bbo[2,o],10))
        plot(0.*linspace(xo[2,o] - bbo[2,o],xo[2,o] + bbo[2,o],10) .+ (xo[1,o] + bbo[1,o]),
            linspace(xo[2,o] - bbo[2,o],xo[2,o] + bbo[2,o],10))
    end
    axis("equal");
end

plotMazeSolution (generic function with 1 method)

In [3]:
# number of waypoints
numWayPoints = 3

# number of obstacles
numObstacles = 1

# number of hovercrafts
numHoverCrafts = 3             

# vector of timepoints
T = zeros(Int,numWayPoints)

# vector of waypoint locations, (x,y) locations of each waypoint
inputWayPoints = zeros(2, numWayPoints, numHoverCrafts)

# vector of obstacle locations, (x,y) locations of each obstacle
inputObstacles = zeros(2, numObstacles)

# bounding box around obstacle, (breadth, length) of the obstacle.
# Breadth is added and subtracted from the x dimension of the location of the obstacle to
# get upper and lower limits of obstacle respectivly.
# Similalry for length.
bbo = zeros(2, numObstacles)

# Maximum velocity at which the hovercraft can travel.
vMax = 1

# Time instant at which the hovercraft should the waypoint
T[1] = 1
T[2] = 20
T[3] = 40


# define waypoints
inputWayPoints[:,:,1] = [-5 10 20
                        8 3 7]

inputWayPoints[:,:,2] = [-5 10 20
                        8 3 7]

inputWayPoints[:,:,3] = [-5 10 20
                        8 3 7]

# define obstacles
inputObstacles = [ 1.5 
                   7.5 ]
    
    
# define bounding box limits of the obstacles
bbo = [ 1.5 
        2.5 ]


#inputObstacles = inputObstacles.*2
#bbo = bbo.*2
#inputObstacles = [inputObstacles.*1.2 inputObstacles[:,2:3].+[15;0] inputObstacles[:,5].+[15;0]]
#bbo = [bbo.*1.2 bbo[:,2:3] bbo[:,5]]
#numObstacles = numObstacles*2

#inputObstacles = inputObstacles[:,1:8]
#bbo = bbo[:,1:8]
#numObstacles = numObstacles - 2

# Matrix to write vectorized constraints for avoiding static obstacles
A = [1 0;
    -1 0;
    0 1;
    0 -1;]

# Matrix to store upper and lower limits of the vectorized constraints
bo = zeros(4, numObstacles)

# This loop constructs b for the vectorized constraints Ax - b <= obstacles
for i in 1:numObstacles
    bo[1,i] = inputObstacles[1,i] - bbo[1,i]
    bo[2,i] = -(inputObstacles[1,i] + bbo[1,i])
    bo[3,i] = inputObstacles[2,i] - bbo[2,i]
    bo[4,i] = -(inputObstacles[2,i] + bbo[2,i])
end

lambda = 100/1000

0.1

In [ ]:
xopt, uopt, objectivevalue, J1, J2 = solveMaze(numWayPoints, numHoverCrafts, numObstacles, A, bo, inputWayPoints, false, false, lambda)
plotMazeSolution(xopt, numWayPoints, numHoverCrafts, numObstacles, A, bo, inputWayPoints, inputObstacles)

In [ ]:
xopt, uopt, objectivevalue, J1, J2 = solveMaze(numWayPoints, numHoverCrafts, numObstacles, A, bo, inputWayPoints, true, false, lambda)
plotMazeSolution(xopt, numWayPoints, numHoverCrafts, numObstacles, A, bo, inputWayPoints, inputObstacles)

In [ ]:
xopt, uopt, objectivevalue, J1, J2 = solveMaze(numWayPoints, numHoverCrafts, numObstacles, A, bo, inputWayPoints, true, true, lambda)
plotMazeSolution(xopt, numWayPoints, numHoverCrafts, numObstacles, A, bo, inputWayPoints, inputObstacles)